In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from models.edenTTS import EdenTTS
from hparams import hparams as hp
from utils.paths import Paths
import torch
import time
import soundfile as sf
import os
from utils.log_util import get_logger
import numpy as np
from text.en_util import text_to_sequence
from qwen_tts import Qwen3TTSTokenizer
import soundfile as sf
# THÊM THƯ VIỆN ĐỂ NGHE AUDIO TRỰC TIẾP TRÊN JUPYTER
from IPython.display import Audio, display 

device = "cuda" if torch.cuda.is_available() else "cpu"
log = get_logger(__name__)

config_file: config/eden.json


2026-02-22 06:26:16,146 INFO en_util.py-25 load symbol2id from resource/symbol_to_id_en.json.
/home/doannv/edentts-with-qwen3-tokenizer/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/bin/sh: 1: sox: not found
2026-02-22 06:26:18,746 WARNING __init__.py-10 SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    



********
********
 


In [3]:
# ==========================================
# 1. LOAD TOKENIZER VÀ MODEL (Chỉ cần chạy 1 lần)
# ==========================================
log.info("Loading Qwen3 TTS Tokenizer...")
tokenizer = Qwen3TTSTokenizer.from_pretrained(
    "Qwen/Qwen3-TTS-Tokenizer-12Hz",
    device_map=device,
)

2026-02-22 06:26:18,967 INFO 2871649631.py-4 Loading Qwen3 TTS Tokenizer...


In [4]:
# Khởi tạo và load trọng số EdenTTS
tts_model_id = hp.tts_model_id
paths = Paths(hp.data_path, tts_model_id, speaker=hp.speaker)
tts_model_path = paths.tts_latest_weights
print(f"Đường dẫn checkpoint: {tts_model_path}")

if not os.path.exists(tts_model_path):
    print(f"⚠️ LỖI: Không tìm thấy file checkpoint tại: {tts_model_path}")
else:
    tts_model = EdenTTS(hp).to(device)
    tts_model.load("/home/doannv/edentts-with-qwen3-tokenizer/checkpoints/ljs/tts_eden/step_20000_weights.pyt")
    tts_model.eval() # Bắt buộc set eval mode
    log.info("EdenTTS Model loaded successfully!")

out_path = "./"
os.makedirs(out_path, exist_ok=True)

Đường dẫn checkpoint: checkpoints/ljs/tts_eden/latest_weights.pyt


/home/doannv/edentts-with-qwen3-tokenizer/.venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
2026-02-22 06:26:21,339 INFO net_utils.py-25  number of weights:9636480, tol params:116, params requires_grad:115
2026-02-22 06:26:21,340 INFO net_utils.py-25  number of weights:77387776, tol params:176, params requires_grad:176
2026-02-22 06:26:21,340 INFO net_utils.py-25  number of weights:591617, tol params:10, params requires_grad:10
2026-02-22 06:26:21,341 INFO net_utils.py-25  number of weights:96795009, tol params:318, params requires_grad:317
2026-02-22 06:26:21,342 INFO edenTTS.py-90 tol_params: 96795009, text_encoder:9636480,decoder:77387776,dur:591617,tol_infer:87615873
2026-02-22 06:26:21,999 INFO 3075912052.py-13 EdenTTS Model loaded successfully!


In [5]:
enc = tokenizer.encode("https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-TTS-Repo/tokenizer_demo_1.wav")
OutputClass = enc.__class__

In [10]:
# ==========================================
# 2. ĐỊNH NGHĨA HÀM SUY LUẬN
# ==========================================
def synthesize_text(text):
    if not os.path.exists(tts_model_path):
        return
        
    log.info(f"Processing: '{text}'")
    phones = text_to_sequence(text, token_type=hp.token_type) 
    phones = torch.tensor(phones).long().unsqueeze(0).to(device)
    
    s1 = time.time()
    # Dự đoán Latent vector liên tục
    mel_pred = tts_model.inference(phones, temperature=1,d_control=1, top_k=5)
    print(mel_pred.shape)
    log.info(f"Acoustic model inference time: {time.time() - s1:.3f}s")
    safe_name = "".join([c if c.isalnum() else "_" for c in text[:15]])
    file_name = f"{safe_name}_{int(time.time())}.wav"
    file_path = os.path.join(out_path, file_name)
    with torch.no_grad():
        # De-normalize: Nhân 2048, làm tròn, ép kiểu integer và kẹp trong khoảng codebook
        tokens = torch.round(mel_pred).long()
        tokens = torch.clamp(tokens, min=0, max=2047)
        print(f"Predicted token shape: {tokens.shape}, token range: [{tokens.min().item()}, {tokens.max().item()}]")
        new_output = OutputClass(audio_codes=[tokens[0]])
        print(new_output)
        # print({"audio_codes": [tokens[0]]})
        
        # Bọc tensor vào dictionary với key "audio_codes"
        wav, sr = tokenizer.decode(new_output)
        print(wav)
        sf.write("test.wav", wav[0], sr)
    
    log.info(f"Saved at: {file_path}")
    
    # Render thanh Audio ngay trên Jupyter Cell
    display(Audio(wav, rate=sr))

In [11]:
# Ô CHẠY THỬ NGHIỆM (Chạy bao nhiêu lần tùy thích)
text_to_test = "Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition"
synthesize_text(text_to_test)

2026-02-22 06:27:44,384 INFO 46541808.py-8 Processing: 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition'
2026-02-22 06:27:44,435 INFO 46541808.py-16 Acoustic model inference time: 0.041s
2026-02-22 06:27:44,598 INFO 46541808.py-34 Saved at: ./Printing__in_th_1771716464.wav


Text Encoder output shape: torch.Size([1, 153, 512])
Expanded Text Value shape: torch.Size([1, 512, 114])
torch.Size([1, 114, 16])
Predicted token shape: torch.Size([1, 114, 16]), token range: [0, 2047]
Qwen3TTSTokenizerV2EncoderOutput(audio_codes=[tensor([[   9,  741, 1744,  ..., 1606, 1926, 2019],
        [ 514, 1746, 1578,  ...,   93, 1675, 1775],
        [ 436, 1373, 1415,  ...,   73, 2013,  300],
        ...,
        [1815, 1787,  755,  ..., 1610,  733, 1332],
        [1815, 1130,  365,  ..., 1776,  167,  223],
        [ 336,  393, 1121,  ...,  389,  220,  983]], device='cuda:0')])
[array([ 5.8088126e-04,  6.6702161e-04,  6.1671028e-04, ...,
       -9.1585280e-06, -2.0636914e-04, -2.8813267e-04],
      shape=(218880,), dtype=float32)]


In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Tuple, Optional
from hparams import hparams
# --- 1. SETTINGS & HELPERS ---

@dataclass
class ModelArgs:
    dim: int = 1024
    n_layers: int = 6
    n_heads: int = 8
    n_kv_heads: Optional[int] = 2
    vocab_size: int = 2048
    multiple_of: int = 256
    ffn_dim_multiplier: Optional[float] = None
    norm_eps: float = 1e-5
    max_batch_size: int = 32
    max_seq_len: int = 2048

@dataclass
# Lấy giá trị trực tiếp từ hparams bạn đã khởi tạo
@dataclass
class DecoderConfig:
    # Lấy từ hparams.text_encoder_hidden (384)
    idim: int = hparams.text_encoder_hidden   
    # Lấy từ hparams.decoder_hidden (512)
    hidden_size: int = hparams.decoder_hidden 
    # Lấy từ hparams.vocab_size (365 - đây là token text, 
    # nhưng nếu bạn dùng cho audio RVQ thì thường là 1024 hoặc 2048)
    vocab_size: int = 2048 
    # Lấy từ hparams.num_mels (16 layers audio)
    num_codebooks: int = hparams.num_mels      
    
    # Layer settings (tùy chỉnh theo kiến trúc Transformer mới)
    num_layers_coarse: int = hparams.decoder_layers # 6
    num_layers_priority: int = 4
    num_layers_shared: int = 2

def sample_token(logits: torch.Tensor, topk: int = 5, temperature: float = 1.0) -> torch.Tensor:
    # Thêm hàm sample đơn giản để code không báo lỗi
    logits = logits / max(temperature, 1e-5)
    v, _ = torch.topk(logits, min(topk, logits.size(-1)))
    pivot = v[..., [-1]]
    logits[logits < pivot] = -float('Inf')
    probs = F.softmax(logits, dim=-1)
    return torch.multinomial(probs.view(-1, probs.size(-1)), 1).view(*probs.shape[:-1])

# --- 2. LLAMA BLOCKS (FROM SCRATCH) ---

class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))
    def forward(self, x):
        return self.weight * (x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)).type_as(x)

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end)
    freqs = torch.outer(t, freqs).float()
    return torch.polar(torch.ones_like(freqs), freqs)

def apply_rotary_emb(xq, xk, freqs_cis):
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = freqs_cis.view(1, xq_.size(1), 1, xq_.size(3))
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    Lặp lại các KV heads để khớp với số lượng Q heads (Cần thiết cho Llama GQA).
    Shape input: (B, T, n_kv_heads, head_dim)
    Shape output: (B, T, n_heads, head_dim)
    """
    if n_rep == 1:
        return x
    bs, sl, n_kv, hd = x.shape
    return (
        x[:, :, :, None, :]
        .expand(bs, sl, n_kv, n_rep, hd)
        .reshape(bs, sl, n_kv * n_rep, hd)
    )


class Attention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.n_kv_heads = args.n_kv_heads if args.n_kv_heads is not None else args.n_heads
        self.head_dim = args.dim // args.n_heads
        
        # Tỷ lệ lặp lại đầu (ví dụ: 16 heads / 2 kv_heads = 8 lần lặp)
        self.n_rep = self.n_heads // self.n_kv_heads
        
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)

    def forward(self, x, freqs_cis):
        bsz, seqlen, _ = x.shape
        xq, xk, xv = self.wq(x), self.wk(x), self.wv(x)

        xq = xq.view(bsz, seqlen, self.n_heads, self.head_dim)
        xk = xk.view(bsz, seqlen, self.n_kv_heads, self.head_dim)
        xv = xv.view(bsz, seqlen, self.n_kv_heads, self.head_dim)

        xq, xk = apply_rotary_emb(xq, xk, freqs_cis=freqs_cis)

        # FIX: Lặp lại KV để khớp với Q trước khi nhân ma trận
        xk = repeat_kv(xk, self.n_rep)
        xv = repeat_kv(xv, self.n_rep)

        # Chuyển về (B, Heads, T, Dim) để matmul
        xq = xq.transpose(1, 2)
        xk = xk.transpose(1, 2)
        xv = xv.transpose(1, 2)

        scores = torch.matmul(xq, xk.transpose(-2, -1)) / (self.head_dim ** 0.5)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        
        output = torch.matmul(scores, xv) # (B, Heads, T, Dim)
        return self.wo(output.transpose(1, 2).contiguous().view(bsz, seqlen, -1))

class FeedForward(nn.Module):
    def __init__(self, dim: int, multiple_of: int):
        super().__init__()
        hidden_dim = int(8 * dim / 3)
        hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        self.w1, self.w2, self.w3 = nn.Linear(dim, hidden_dim, bias=False), nn.Linear(hidden_dim, dim, bias=False), nn.Linear(dim, hidden_dim, bias=False)
    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

class TransformerBlock(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.attention = Attention(args)
        self.feed_forward = FeedForward(args.dim, args.multiple_of)
        self.attention_norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.ffn_norm = RMSNorm(args.dim, eps=args.norm_eps)
    def forward(self, x, freqs_cis):
        x = x + self.attention(self.attention_norm(x), freqs_cis)
        return x + self.feed_forward(self.ffn_norm(x))

class LlamaTransformer(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.args = args
        self.layers = nn.ModuleList([TransformerBlock(args) for _ in range(args.n_layers)])
        self.norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.tok_embeddings = nn.Embedding(args.vocab_size, args.dim)
        self.output = nn.Linear(args.dim, args.vocab_size, bias=False)
        self.register_buffer("freqs_cis", precompute_freqs_cis(args.dim // args.n_heads, args.max_seq_len))

    def forward(self, x):
        # x is embedding if tok_embeddings is Identity
        freqs_cis = self.freqs_cis[:x.shape[1]].to(x.device)
        for layer in self.layers:
            x = layer(x, freqs_cis)
        return self.norm(x)

def get_llama3_2_from_scratch(num_layers, num_heads, embed_dim):
    args = ModelArgs(n_layers=num_layers, n_heads=num_heads, dim=embed_dim)
    return LlamaTransformer(args)

def prepare_transformer(model):
    model.tok_embeddings = nn.Identity()
    model.output = nn.Identity()
    return model, model.args.dim

# --- 3. THE DECODER V3 (UPDATED) ---

class TransformerDecoderV3(nn.Module):
    def __init__(self, config: DecoderConfig):
        super().__init__()
        self.config = config
        
        # Dùng hàm "from_scratch" thay vì thư viện ngoài
        self.coarse_decoder, _ = prepare_transformer(
            get_llama3_2_from_scratch(num_layers=config.num_layers_coarse, num_heads=16, embed_dim=config.hidden_size)
        )
        self.coarse_head = nn.Linear(config.hidden_size, config.vocab_size)

        self.priority_decoders = nn.ModuleList([
            prepare_transformer(get_llama3_2_from_scratch(num_layers=config.num_layers_priority, num_heads=8, embed_dim=config.hidden_size))[0]
            for _ in range(3)
        ])
        
        self.shared_fine_decoder, _ = prepare_transformer(
            get_llama3_2_from_scratch(num_layers=config.num_layers_shared, num_heads=8, embed_dim=config.hidden_size)
        )

        self.audio_embeddings = nn.Embedding(config.vocab_size * config.num_codebooks, config.hidden_size)
        self.register_buffer("audio_shifts", (config.vocab_size * torch.arange(config.num_codebooks)).view(1, 1, -1))

        self.fine_heads = nn.ModuleList([nn.Linear(config.hidden_size, config.vocab_size) for _ in range(config.num_codebooks - 1)])
        self.fusion_layers = nn.ModuleList([nn.Linear(config.hidden_size * 2, config.hidden_size) for _ in range(config.num_codebooks - 1)])

    def _get_layered_embedding(self, tokens: torch.Tensor, layer_idx: int):
        shifted_tokens = tokens + (layer_idx * self.config.vocab_size)
        return self.audio_embeddings(shifted_tokens)

    def forward(self, text_features: torch.Tensor, targets: Optional[torch.Tensor] = None, temperature: float = 0.8):
        B, T, _ = text_features.shape
        logits_list = []
        
        h_coarse = self.coarse_decoder(text_features) 
        logits_0 = self.coarse_head(h_coarse)
        logits_list.append(logits_0.unsqueeze(2)) 

        current_tokens = targets[:, :, 0] if targets is not None else sample_token(logits_0, temperature=temperature)
        current_hidden = h_coarse

        for i in range(self.config.num_codebooks - 1):
            emb_prev = self._get_layered_embedding(current_tokens, layer_idx=i)
            current_hidden = self.fusion_layers[i](torch.cat([current_hidden, emb_prev], dim=-1))

            h = self.priority_decoders[i](current_hidden) if i < 3 else self.shared_fine_decoder(current_hidden)
            
            logits_i = self.fine_heads[i](h)
            logits_list.append(logits_i.unsqueeze(2))
            
            if i < self.config.num_codebooks - 2:
                current_tokens = targets[:, :, i+1] if targets is not None else sample_token(logits_i, temperature=temperature)
            current_hidden = h

        return torch.cat(logits_list, dim=2) # Shape: (B, T, num_codebooks, vocab)

In [58]:
config = DecoderConfig()
t_model = TransformerDecoderV3(config)
dummy_text = torch.randn(2, 50, 512) # Batch 2, Seq 50, Dim 1024
output = t_model(dummy_text)
print(output.shape) # Kì vọng: [2, 50, 16, 2048]

torch.Size([2, 50, 16, 2048])


In [57]:
from torchinfo import summary

# Khởi tạo config từ hparams của bạn
config = DecoderConfig()
t_model = TransformerDecoderV3(config)

# Chỉ định input_size theo đúng format (Batch, Seq_Len, Hidden_Dim)
# Bạn không cần truyền số '2' vào trong tuple nếu dùng batch_dim=0
summary(t_model, input_size=(2, 50, hparams.text_encoder_hidden))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []